In [ ]:
server_ip    = '0.0.0.0' 
control_port = 54322
sensor_port  = 54324
flask_port   = 54323 
basic_auth_username = '31*hjbhv(41ok$'
basic_auth_password = '09&nb52#mcolz;'
video_fps=25

In [ ]:
import os, ssl
server_cert_letsencrypt  = f'{os.environ["HOME"]}/openssl/fullchain1.pem'  # secret of my apache and flask site
server_key_letsencrypt   = f'{os.environ["HOME"]}/openssl/privkey1.pem'    # secret of my apache and flask site
server_cert_self_signed  = 'setup/secret/server.crt'    # shared with both rover and driver for identity authentication of server 
server_key_self_signed   = 'setup/secret/server.key'    
client_certs_self_signed = 'setup/secret/clients.crt'   # for identity authentication of rover and driver
ssl_context_self_signed = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH, cafile=client_certs_self_signed)  
ssl_context_self_signed.verify_mode = ssl.CERT_REQUIRED
ssl_context_self_signed.load_cert_chain(certfile=server_cert_self_signed, keyfile=server_key_self_signed) 

## Backend Sensor Logging Database

In [ ]:
import time, dateutil, json, queue, sqlite3, threading 

database_queue_in, database_queue_out = queue.Queue(), queue.Queue() 
def database_daemon(): # avoid sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread.
    EST = dateutil.tz.gettz('America/New York') # 'UTC'
    db = sqlite3.connect(":memory:")
    #try:
    #    os.remove("GPS.db")
    #    db = sqlite3.connect("GPS.db")
    #except:
    #    pass
    cur = db.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ('tracking',))
    if not cur.fetchone():
        cur.execute("""CREATE TABLE tracking (gps_valid, temperature, x, y, z, voltage, latitude, longitude, altitude, course, speed, satellite, rover_localtime, readable_rover_localtime, server_localtime)""") 
    while True: 
        try:
            item = database_queue_in.get() 
            if item[0] == 'INSERT': 
                js = json.loads(item[1]) 
                if js['gps']: 
                    ts = dateutil.parser.isoparse(js['ts'])   
                    cur.execute("""INSERT INTO tracking (gps_valid, temperature,      x,      y,      z, voltage, latitude, longitude, altitude,   course,    speed, satellite, rover_localtime,                               readable_rover_localtime, server_localtime) 
                                                 VALUES (        ?,           ?,      ?,      ?,      ?,       ?,        ?,         ?,        ?,        ?,        ?,         ?,               ?,                                                      ?,                ?)""", 
                                                        (     True,     js['T'],js['x'],js['y'],js['z'],js['v'] ,js['lat'], js['lon'],js['lat'],js['cus'],js['spd'], js['sat'],  ts.timestamp(), ts.astimezone(EST).strftime("%Y-%m-%d %H:%M:%S (EST)"),      time.time()))  
                else:
                    cur.execute("""INSERT INTO tracking (gps_valid, temperature,      x,      y,      z, voltage,                                                                                                                                        server_localtime) 
                                                 VALUES (        ?,           ?,      ?,      ?,      ?,       ?,                                                                                                                                                       ?)""", 
                                                        (    False,     js['T'],js['x'],js['y'],js['z'],js['v'] ,                                                                                                                                             time.time()))   
            elif item[0] == 'newcomer':  #                    earliest gps positition - Home Spot
                cur.execute("""SELECT longitude, latitude, readable_rover_localtime, server_localtime FROM tracking 
                               WHERE server_localtime=(SELECT MIN(server_localtime) FROM tracking WHERE gps_valid==1)""") 
                database_queue_out.put(cur.fetchone()) 
            elif item[0] == 'trail':     #                    Trail   all valid record
                cur.execute("""SELECT longitude, latitude, readable_rover_localtime FROM tracking 
                               WHERE server_localtime>=? AND gps_valid==1""", (float(item[1]),))    
                database_queue_out.put(cur.fetchall())  
            elif item[0] == 'telemeter': #                    Trail   all valid record
                cur.execute("""SELECT temperature, x, y, z, voltage FROM tracking
                               WHERE server_localtime=(SELECT MAX(server_localtime) FROM tracking)""")    
                database_queue_out.put(cur.fetchone())
            elif item[0] == 'debug':     
                cur.execute(item[1]) 
                database_queue_out.put(cur.fetchall()) 
            database_queue_in.task_done()
            #if item[0]!='INSERT': print(item) 
        except Exception as e:
            print(e,'\t|\t\t\t\t|\t', item)
threading.Thread(target=database_daemon).start()  

def test_db():
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.1,"lon":-2.3,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:41Z","gps":1}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.2,"lon":-2.4,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:42Z","gps":0}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.3,"lon":-2.5,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:45Z","gps":0}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.4,"lon":-2.6,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:47Z","gps":1}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.14,"z":-0.56,"v":0.03,"lat":1.5,"lon":-2.7,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:49Z","gps":1}'
    database_queue_in.put(('INSERT', s)) 
    
    database_queue_in.put(('debug', "SELECT * FROM tracking;")) 
    print(database_queue_out.get())
    print('\n\n\n')
    
    database_queue_in.put(('telemeter', )) 
    temperature, x, y, z, voltage = database_queue_out.get() 
    print(temperature, x, y, z, voltage, '\n\n\n')
    
    database_queue_in.put(('newcomer', )) 
    longitude, latitude, rover_localtime, oldest_server_localtime = database_queue_out.get()  
    print(longitude, latitude, rover_localtime, oldest_server_localtime, '\n\n\n') 
    
    database_queue_in.put(('trail', oldest_server_localtime)) 
    print(database_queue_out.get()) 
    
    database_queue_in.put(('debug', "DELETE FROM tracking;")) 
#test_db()

## Sensor Link

In [ ]:
import struct, socket 
def sensor_link_daemon():
    long  = struct.calcsize('<H')
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((server_ip, sensor_port))
    s.listen(0)
    while True: 
        print(f"[Sensor link - Rover] Waiting for incoming connection")
        c, a = s.accept() 
        print(f"[Sensor link - Rover] Connection established")
        ss = ssl_context_self_signed.wrap_socket(c, server_side=True).makefile('rb')
        while True:
            try:
                data_len, = struct.unpack('<H', ss.read(long)) 
                data = ss.read(data_len) 
                database_queue_in.join()
                #print(data)
                database_queue_in.put(('INSERT', data)) 
            except socket.timeout:  
                ss.shutdown(socket.SHUT_RDWR)  
                ss.close()
                print('[Sensor link - Rover] timeout')
                break
            except ConnectionResetError: 
                print('[Sensor link - Rover] ConnectionResetError')
                break
sensor_link_thread  = threading.Thread(target=sensor_link_daemon); sensor_link_thread.start(); print('sensor_link_daemon', sensor_link_thread.getName())

## Control Link

In [ ]:
def getCommonName(ss):
    for i in ss.getpeercert()['subject']:
        if i[0][0]=='commonName':
            return i[0][1] 
        
command_queue = queue.Queue()    # cache the command to be forwarded

def command_receiver_daemon(ss): 
    l = struct.calcsize('<Bff')
    while True:
        try:
            data = ss.recv(l)  
            command_queue.put(data)  
        except socket.timeout:  
            ss.shutdown(socket.SHUT_RDWR)  
            ss.close()
            print('[Control link - Driver] Timeout')
            break
        except ConnectionResetError: 
            print('[Control link - Driver] ConnectionResetError')
            break
        
def command_transmitter_daemon(ss): 
    while True: 
        data = command_queue.get()
        try:  
            ss.send(data) # Send code and value to server 
        except socket.timeout:
            ss.shutdown(socket.SHUT_RDWR)  
            ss.close() 
            print('[Control link - Rover] Timeout')
            break
        except BrokenPipeError: 
            print('[Control link - Rover] BrokenPipeError')
            break
        except ConnectionResetError: 
            print('[Control link - Rover] ConnectionResetError')
            break

def control_link_daemon(): 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((server_ip, control_port))
    s.listen(0)
    while True: 
        print(f"[Control link] Waiting for incoming connection")
        c, a = s.accept() 
        ss = ssl_context_self_signed.wrap_socket(c, server_side=True)
        ss.settimeout(30)
        name = getCommonName(ss)
        print(f"[Control link] Connection with {name} established ~")
        if name == 'driver': 
            threading.Thread(target=command_receiver_daemon   , args=(ss,)).start()
        elif name == 'rover':
            threading.Thread(target=command_transmitter_daemon, args=(ss,)).start() 

control_link_thread = threading.Thread(target=control_link_daemon); control_link_thread.start(); print('control_link_daemon', control_link_thread.getName())

## Image Link
pip install opencv-python 

In [ ]:
import cv2
class MJPEG():# image cache
    def __init__(self):  
        self.frame = b''
        threading.Thread(target=self.updater).start()
         
    def updater(self):
        cap = cv2.VideoCapture("tcp://127.0.0.1:2222") 
        print('[Image link] Start capturing')
        while(cap.isOpened()):   
            ret, frame = cap.read() 
            if not ret:
                # print('continue')
                continue 
            ret, jpeg  = cv2.imencode('.jpg', frame)
            self.frame = jpeg.tobytes() 
            
mjpeg = MJPEG()

## Web Front
pip install flask flask_basicauth flask_cors 

In [ ]:
import flask, flask_basicauth, flask_cors 
with open('src/map.html', 'r') as fr: mapPage = fr.read()
app = flask.Flask(__name__)
flask_cors.CORS(app) 
app.config['BASIC_AUTH_USERNAME'] = basic_auth_username
app.config['BASIC_AUTH_PASSWORD'] = basic_auth_password
basic_auth = flask_basicauth.BasicAuth(app)
#log = logging.getLogger('werkzeug')
#log.setLevel(logging.ERROR) 
interval = 1/video_fps
@app.route("/")
@basic_auth.required
def home(): 
    return flask.render_template_string(mapPage, 
        ip_port       = f'{server_ip}:{flask_port}', 
        username      =  basic_auth_username, 
        password      =  basic_auth_password)  
    
@app.route("/imageready", methods=['GET'])
@basic_auth.required
def imageready(): 
    if mjpeg.frame:
        return flask.jsonify({"imageready":True}) 
    else:
        return flask.jsonify({"imageready":False}) 

@app.route("/stream.mjpg")
@basic_auth.required
def stream():  
    def generator():      
        while True:     
            time.sleep(interval)  # threading.condition is too shitty according to my test. no condition no lag. 
            frame = mjpeg.frame  
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

def read_sql(put): 
    database_queue_in.join()
    database_queue_in.put(put)
    get = q_out.get()
    q_out.task_done()  
    return get 

@app.route("/telemeter", methods=['GET'])
@basic_auth.required
def telemeter():
    _ = read_sql(("telemeter", ))
    if _: 
        temp, y, z, v = _  
        RESPONSE={'telemeterReady':True, 'temp':temp, 'y':y, 'z':z, 'v':v}
    else:
        RESPONSE={'telemeterReady':False} 
    return flask.jsonify(RESPONSE)
    
@app.route("/gps", methods=['GET'])
@basic_auth.required
def gps():
    newestTimestampAtClientSide = flask.request.args.get('q')
    if newestTimestampAtClientSide == 'newcomer':
        _ = read_sql(("newcomer", ))
        if _: # There is valid GPS data
            lon, lat, localtime, newestTimestampAtServerSide = _  
            RESPONSE={ 
                "HomeSpot": [lat, lon], # lat lon in Google and Openstreetmap
                "newestTimestampAtServerSide": newestTimestampAtServerSide 
            }
        else:
            RESPONSE={"newestTimestampAtServerSide": "newcomer"}
    else: # newestTimestampAtClientSide is latest timestamp
        _ = read_sql(('trail', newestTimestampAtClientSide))
        if _: #
            lon, lat, localtime, newestTimestampAtServerSide, speed, x = _[-1] 
            if float(newestTimestampAtClientSide) == newestTimestampAtServerSide: # no new timestamp from database
                RESPONSE={"newestTimestampAtServerSide": newestTimestampAtClientSide}
            else: 
                RESPONSE={
                    "trailReady": True,
                    "CurrentSpot": [lat, lon], # lat lon in Google and Openstreetmap
                    "speed": speed,
                    "x": x,
                    "newestTimestampAtServerSide": newestTimestampAtServerSide, 
                    "trail": {
                        "type": "Feature",
                        "geometry": {
                            "type": "LineString",
                            "coordinates": [i[:2] for i in _]
                },},}
        else:
            RESPONSE={"trailReady": False}
    return flask.jsonify(RESPONSE)

@app.route('/axios.min.js', methods=['GET'])
@basic_auth.required
def axios_js():
        return flask.send_file('src/axios.min.js', mimetype='application/javascript')
    
@app.route('/leaflet.rotatedMarker.js', methods=['GET'])
def rotatedMarker_js():
        return flask.send_file('src/leaflet.rotatedMarker.js', mimetype='application/javascript')
     
@app.route('/rainbow.jpg', methods=['GET'])
def rainbow_svg():
        return flask.send_file('src/rainbow.jpg', mimetype='image/jpeg')
    
@app.route('/home.png', methods=['GET'])
def home_svg():
        return flask.send_file('src/home.png', mimetype='image/png')
    
@app.route('/rover.svg', methods=['GET'])
def arrow_svg():
        return flask.send_file('src/rover.svg', mimetype='image/svg+xml')

def flask_in_bg():
    app.run(server_ip, flask_port, ssl_context=(server_cert_letsencrypt, server_key_letsencrypt)) 

In [ ]:
print('https://guoxiaokang.net:54323/stream.mjpg')
flask_in_bg()